<a href="https://colab.research.google.com/github/Dntfreitas/introduction-agents-ai/blob/main/5_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Function Calling in OpenAI

OpenAI's **function calling** feature allows language models like GPT-4 to interact with external tools, APIs, or Python functions in a structured and predictable way. This is particularly useful when you want the model to trigger specific actions, retrieve up-to-date data, or follow a controlled workflow.

#### What is Function Calling?

Instead of producing free-form text, the model can be instructed to return structured data that matches the signature of a predefined function. The model can "decide" to call one of these functions and generate the appropriate arguments, allowing developers to safely and reliably integrate AI into complex applications.

#### How It Works

1. **Define a function schema**: You provide the model with the name, description, and parameters of the function using a JSON schema.
2. **Send a prompt and functions to the model**: Along with the user's message, you send the list of function definitions.
3. **Model responds with a function call**: If appropriate, the model will return the name of the function and the arguments it believes should be used.
4. **Execute the function in your environment**: Your code runs the function using the provided arguments.
5. **Optionally send the result back to the model**: The output of the function can be fed back into the model for further reasoning or response generation.

In [ ]:
# Let's make sure we have the required libraries installed for this tutorial.
!pip install openai dateparser gradio requests PyPDF2

In [ ]:
# Now, let's import the necessary libraries and set up our environment.

import json

import dateparser
import gradio as gr
import requests
from PyPDF2 import PdfReader
from openai import OpenAI


In [ ]:
# As we are going to use Google Coolab, we don't need to load the environment variables.
# Otherwise, you can use the following code to load the environment variables from a `.env` file.
# from dotenv import load_dotenv
# load_dotenv(override=True)

from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
# Now, let's initialize the OpenAI API client.
openai = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# Load and Extract Madeira PDF Content
# This is an easier, less efficient, RAG method.

reader = PdfReader("docs/Madeira.pdf")
pdf_text = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        pdf_text += text

In [ ]:
# Create the System Prompt
# The prompt defines the role of the assistant: a knowledgeable and professional tourist guide.

system_prompt = (
    "You are acting as a tourist guide in Madeira. You are answering questions on a website, "
    "about the island of Madeira, particularly questions related to the island's history, culture, "
    "and tourist attractions. Your responsibility is to represent Madeira for interactions on the "
    "website as faithfully as possible. Be professional and engaging, as if talking to a potential "
    "client or future employer who came across the website. If you don't know the answer, say so.\n\n"
)
system_prompt += f"## Here is a PDF with information about Madeira:\n{pdf_text}\n\n"
system_prompt += "With this context, please chat with the user, always staying in character as a tourist guide in Madeira."
system_prompt += " Please, make sure to answer only with the information provided in the PDF. Also, please match the language of the user."


In [ ]:
# Define utility Functions

def check_forecast(globalIdLocal, forecastDate):
    """
    Gets the weather forecast for a specific location and date using IPMA API.
    """
    url = f"https://api.ipma.pt/open-data/forecast/meteorology/cities/daily/{globalIdLocal}.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        for day in data['data']:
            if day['forecastDate'] == forecastDate:
                return day
    else:
        print(f"Error fetching forecast: {response.status_code}")
        return None


def get_date_formatted(forecastDate):
    """
    Converts a natural language date into a formatted string (YYYY-MM-DD).
    """
    parsed_date = dateparser.parse(forecastDate)
    if not parsed_date:
        return {"error": "Could not parse the date. Try a format like 'tomorrow' or 'next Friday'."}
    return parsed_date.strftime("%Y-%m-%d")


def get_globalIdLocal():
    """
    Returns the `globalIdLocal` for Funchal from the IPMA region list.
    """
    url = "https://api.ipma.pt/open-data/distrits-islands.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        for region in data['data']:
            if region['local'] == 'Funchal':
                return region['globalIdLocal']
    else:
        print(f"Error fetching location ID: {response.status_code}")
        return None


In [ ]:
# Define the tool functions for structured use

check_forecast_json = {
    "name": "check_forecast",
    "description": "Gets the weather forecast for a given date.",
    "parameters": {
        "type": "object",
        "properties": {
            "globalIdLocal": {
                "type": "number",
                "description": "The location ID to get the weather forecast for."
            },
            "forecastDate": {
                "type": "string",
                "description": "Date of the forecast in 'YYYY-MM-DD' format."
            }
        },
        "required": ["globalIdLocal", "forecastDate"],
        "additionalProperties": False,
    }
}

get_date_formatted_json = {
    "name": "get_date_formatted",
    "description": "Parses a natural language date into a standard format.",
    "parameters": {
        "type": "object",
        "properties": {
            "forecastDate": {
                "type": "string",
                "description": "Natural language input (e.g. 'next Friday')."
            }
        },
        "required": ["forecastDate"],
        "additionalProperties": False,
    }
}

get_global_id_json = {
    "name": "get_globalIdLocal",
    "description": "Gets the globalIdLocal for Funchal, Madeira.",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False,
    }
}

In [ ]:
tools = [
    {"type": "function", "function": check_forecast_json},
    {"type": "function", "function": get_global_id_json},
    {"type": "function", "function": get_date_formatted_json}
]

In [ ]:
# Tool execution Engine

def handle_tool_calls(tool_calls):
    """
    Executes tools requested by the AI and returns results.
    """
    results = []

    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        if tool_name == "check_forecast":
            result = check_forecast(**arguments)
        elif tool_name == "get_date_formatted":
            result = get_date_formatted(**arguments)
        elif tool_name == "get_globalIdLocal":
            result = get_globalIdLocal()
        else:
            raise ValueError(f"Unknown tool name: {tool_name}")

        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })

    return results

In [ ]:
# Chat function to be used with Gradio

def chat(message, history):
    """
    Manages the chat interaction and handles tool use if the model requests it.
    """
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False

    while not done:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools
        )

        finish_reason = response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            print(f"Tool call detected: {tool_calls}")
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()